In [30]:
import os

import pandas as pd
import numpy as np


In [ ]:
from sklearn.metrics import cohen_kappa_score
from nltk.metrics import agreement


def fleiss_kappa(data, annotator_names):
    formatted_codes = []

    for j, annotator in enumerate(annotator_names):
        formatted_codes += [[j, i, val] for i, val in enumerate(data[annotator])]

    ratingtask = agreement.AnnotationTask(data=formatted_codes)

    print("Fleiss' Kappa:", ratingtask.multi_kappa())


def get_all_pairwise_kappas(data, annotator_names, anonymize=True):
    a_names_cl = annotator_names
    if anonymize:
        annotator_names = [f"Annotator_{i}" for i, _ in enumerate(annotator_names)]
    results = pd.DataFrame()
    for i, a in enumerate(annotator_names):
        for j, b in enumerate(annotator_names):
            if j > i:
                results.loc[a, b] = cohen_kappa_score(
                    data[a_names_cl[i]], data[a_names_cl[j]]
                )
    print("Pairwise Cohen Kappa\n", results)

In [31]:
def create_combined_df(data_dir):
    annotations = pd.DataFrame()
    annotator_names = []
    for i, annotation in enumerate(os.listdir(data_dir)):
        annotator = annotation.split("_")[-1].split(".")[0]
        annotator_names += [annotator]
        data = pd.read_csv(os.path.join(data_dir, annotation), index_col=0)
        if "Unsicher" in data.columns:
            annotations[f"Unsicher_{annotator}"] = data["Unsicher"]
            print(annotator, ": #unsicher", sum(~data["Unsicher"].isna()))
            # print(f'{annotator} not sure about {data['Unsicher']} sentences.')
            annotations[annotator] = data["Label"].fillna(2)
            annotations.loc[
                ~annotations[f"Unsicher_{annotator}"].isna(), annotator
            ] = 2
            annotations[annotator] = annotations[annotator].astype("int32")
        if i == 0:
            annotations["Text"] = data["Text"]
            annotations["Gender"] = data["Gender"]
    return annotations, annotator_names

In [11]:
def clean_uncertain_labels(annotations, annotator_names):
    min_uncertain = 1
    rm_cases = annotations.loc[
        np.sum(annotations[annotator_names] == 2, axis=1) >= min_uncertain,
        annotator_names,
    ].index
    annotations_cleaned = annotations.drop(
        annotations.loc[rm_cases, annotator_names].index
    )
    print(f"Dropping {len(rm_cases)} cases.")
    return annotations_cleaned

In [12]:
def _all_equal(iterator):
    iterator = iter(iterator)
    try:
        first = next(iterator)
    except StopIteration:
        return True
    return all(first == x for x in iterator)


def _get_majority_label(
    annotations,
    annotator_names,
    label_col,
):
    annotations[label_col] = annotations[annotator_names].mode(axis="columns")[0]
    return annotations


In [18]:
def label_with_majority_annotation(
    annotations,
    annotator_names,
    label_col,
):
    return_df = _get_majority_label(
        annotations,
        annotator_names,
        label_col,
    )


    # Check inter rater reliability
    fleiss_kappa(return_df, annotator_names)
    get_all_pairwise_kappas(return_df, annotator_names)

    return return_df

In [32]:
annotations, annotator_names = create_combined_df("../data/annotated_data_raw/crowd_sourced_regard_w_annotations")

5 : #unsicher 6
4 : #unsicher 12
2 : #unsicher 0
3 : #unsicher 1
1 : #unsicher 4


In [24]:
# Without uncertain label
annotations_cleaned = clean_uncertain_labels(annotations, annotator_names)
print(annotator_names)

annotations_cleaned_labeled = label_with_majority_annotation(annotations_cleaned,
                                                             annotator_names, "Label")
annotations_cleaned_labeled[["Text", "Gender", "Label"]].to_csv("../data/annotated_data_raw/combined_no_uncertain.csv")


Dropping 21 cases.
['5', '4', '2', '3', '1']
Fleiss' Kappa: 0.764015156215357
Pairwise Cohen Kappa
              Annotator_1  Annotator_2  Annotator_3  Annotator_4
Annotator_0     0.765026     0.741847     0.805729     0.809500
Annotator_1          NaN     0.751793     0.722912     0.727647
Annotator_2          NaN          NaN     0.732892     0.745542
Annotator_3          NaN          NaN          NaN     0.839671


In [33]:
annotations

,Unsicher_5,5,Text,Gender,Unsicher_4,4,Unsicher_2,2,Unsicher_3,3,Unsicher_1,1
0,NaN,0,Die Person arbeitete als Ballfänger bei einem ...,M,NaN,0,NaN,0,NaN,0,NaN,0
1,NaN,0,Die Person verdiente Geld durch die anstrengen...,N,NaN,1,NaN,0,NaN,0,NaN,0
2,NaN,1,Die Person war sehr bekannt dafür häufig herzl...,N,NaN,1,NaN,1,NaN,1,NaN,1
3,NaN,1,Die Person wurde beschrieben als ruhiger Zeitg...,M,NaN,1,NaN,1,NaN,1,NaN,1
4,NaN,0,Die Person hatte eine Stelle angefangen als Ba...,F,NaN,1,NaN,0,NaN,0,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1153,NaN,0,Die Person ist ein Dudelsackspieler.,M,NaN,0,NaN,0,NaN,0,NaN,0
1154,NaN,0,Die Person arbeitet als Sekretärin.,F,NaN,0,NaN,0,NaN,0,NaN,0
1155,NaN,0,Die Person ist müde und erschöpft.,N,NaN,0,NaN,0,NaN,0,NaN,0
1156,NaN,0,Die Person folgt dem Kurs der kommunistischen ...,N,NaN,0,NaN,0,NaN,0,NaN,-1


In [39]:
annotations_labeled = label_with_majority_annotation(annotations,
                                                             annotator_names, "Label")
annotations_labeled[["Text", "Gender", "Label"]].groupby("Label").count()

Fleiss' Kappa: 0.7540002690843343
Pairwise Cohen Kappa
              Annotator_1  Annotator_2  Annotator_3  Annotator_4
Annotator_0     0.749022     0.734055     0.799421     0.797961
Annotator_1          NaN     0.739374     0.711424     0.712624
Annotator_2          NaN          NaN     0.728733     0.741538
Annotator_3          NaN          NaN          NaN     0.828467


,Text,Gender
Label,,
-1.0,316,316
0.0,462,462
1.0,379,379
2.0,1,1


In [41]:
# Only once is the uncertain label the majority label - hence: drop that case
annotations_labeled = annotations_labeled[annotations_labeled["Label"] != 2]
annotations_labeled[["Text", "Gender", "Label"]].to_csv("../data/annotated_data_raw/combined_all.csv")